<a href="https://colab.research.google.com/github/marcoswds/sentiment-analysis/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

df = pd.read_csv('imdb.csv',sep=';')

In [ ]:
df.head()

,"review,sentiment",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,One of the other reviewers has mentioned that ...,NaN,NaN,NaN,NaN,NaN,NaN
1,A wonderful little production. <br /><br />The...,NaN,NaN,NaN,NaN,NaN,NaN
2,I thought this was a wonderful way to spend ti...,NaN,NaN,NaN,NaN,NaN,NaN
3,Basically there's a family where a little boy ...,NaN,NaN,NaN,NaN,NaN,NaN
4,"Petter Mattei's ""Love in the Time of Money"" is...",NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#dropa colunas importadas erronemanete
df.drop(['Unnamed: 1','Unnamed: 2','Unnamed: 3','Unnamed: 4','Unnamed: 5','Unnamed: 6'],axis=1,inplace=True)

In [ ]:
#renomeia a coluna importada para o nome correto
df.rename(columns={'review,sentiment': 'review'},inplace=True)

In [ ]:
#cria uma coluna do sentimento pegando a informação da ultima virgula até o final
df['sentiment'] = df['review'].apply( lambda x: x.split(',')[-1])

In [ ]:
#retira a informação do sentimento da coluna review
df['review'] = df['review'].apply( lambda x: x[:-1*(len(x.split(',')[-1]))])

In [ ]:
def limpar_lixo(x:str) -> str:

  lixos = [
    '<br /><br />'
  ]

  for lixo in lixos:
    x = x.replace(lixo,'')

  return x  

df['review'] = df['review'].apply( lambda x: limpar_lixo(x))

In [ ]:
import nltk
nltk.download(["stopwords","punkt","vader_lexicon"])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
from nltk.tokenize import word_tokenize

df['tokenized_text'] = df['review'].apply(word_tokenize) 

In [ ]:
from nltk.corpus import stopwords

stopwords = stopwords.words('english')
df['tokenized_text'] = df['tokenized_text'].apply(lambda x: [elem for elem in x if elem not in stopwords ]) 

In [ ]:
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer("english")

df['tokenized_text'] = df['tokenized_text'].apply(lambda x: [stemmer.stem(elem) for elem in x ])

In [ ]:
import numpy as np 
lst_col = 'tokenized_text'

#Criando um row para cada palavra de cada lista do tokenized_text
df_words = pd.DataFrame({
      col:np.repeat(df[col].values, df[lst_col].str.len())
      for col in df.columns.drop(lst_col)}
    ).assign(**{lst_col:np.concatenate(df[lst_col].values)})[df.columns]

In [ ]:
#Cria um index de 0 pra negativo e 1 para positivo de acordo com o sentimento
df_words['sentiment_index'] = df_words['sentiment'].apply(lambda x: 1 if x == 'positive' else 0)
#A coluna sentiment_index possui agora um valor variando de 0 a 1  para indicar se ela está mais relacionada com valores positivos ou negativos
df_words = df_words.groupby(['tokenized_text'], as_index=False).agg({'sentiment_index': [np.mean]}) 
df_words.columns = list(map(''.join, df_words.columns.values))

#Cria um dicionário do sentiment index para ser mais fácil a pesquisa
dict_tst = df_words.set_index('tokenized_text')['sentiment_indexmean'].to_dict()

Muito da modelagem daqui para frente foi baseada em:
https://realpython.com/python-nltk-sentiment-analysis/#training-and-using-a-classifier

In [ ]:
from statistics import mean

def find_positive_score(word):
    try:
      positive_score = dict_tst[word]
    except:
      positive_score = None
    
    return positive_score

#função para usar com frases soltas
def extract_features(text):
    limpar_lixo(text)
    words = word_tokenize(text)
    words = [elem for elem in words if elem not in stopwords]
    words = [stemmer.stem(elem) for elem in words ]
    features = dict()
    wordcount = 0
    positive_scores = list()

    for word in words:
      if not find_positive_score(word)==None:
        positive_scores.append(find_positive_score(word))

    features["mean_positive"] = mean(positive_scores)

    return features

#função para usar com tudo tratado no dataframe
def extract_features_ready(words):
    features = dict()
    positive_scores = list()

    for word in words:
      if not find_positive_score(word)==None:
        positive_scores.append(find_positive_score(word))

    features["mean_positive"] = mean(positive_scores)

    return features    

In [ ]:
features = [
    (extract_features_ready(review), "pos")
    for review in df[df['sentiment'] == 'positive'].tokenized_text.tolist() 
]
features.extend([
    (extract_features_ready(review), "neg")
    for review in df[df['sentiment'] == 'negative'].tokenized_text.tolist() 
])

In [ ]:
from sklearn.naive_bayes import (
    BernoulliNB,
    ComplementNB,
    MultinomialNB,
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from random import shuffle

classifiers = {
    "BernoulliNB": BernoulliNB(),
    "ComplementNB": ComplementNB(),
    "MultinomialNB": MultinomialNB(),
    "KNeighborsClassifier": KNeighborsClassifier(),
    "DecisionTreeClassifier": DecisionTreeClassifier(),
    "RandomForestClassifier": RandomForestClassifier(),
    "LogisticRegression": LogisticRegression(),
    "MLPClassifier": MLPClassifier(max_iter=1000),
    "AdaBoostClassifier": AdaBoostClassifier(),
}

train_count = len(features) // 4
shuffle(features)
for name, sklearn_classifier in classifiers.items():
  classifier = nltk.classify.SklearnClassifier(sklearn_classifier)
  classifier.train(features[:train_count])
  accuracy = nltk.classify.accuracy(classifier, features[train_count:])
  print(F"{accuracy:.2%} - {name}")

49.93% - BernoulliNB
49.93% - ComplementNB
49.93% - MultinomialNB
92.11% - KNeighborsClassifier
89.64% - DecisionTreeClassifier
89.64% - RandomForestClassifier
92.82% - LogisticRegression
92.83% - MLPClassifier
92.82% - AdaBoostClassifier


In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

def calculate_sia_score(review):
  sia_score = sia.polarity_scores(review)
  if sia_score['pos'] >= sia_score['neg']:
    return 'positive'
  else:
    return 'negative'

df['sia_sentiment'] = df['review'].apply(lambda x:calculate_sia_score(x)) 
df['sia_index'] = df.apply(lambda x: 1 if x['sia_sentiment'] == x['sentiment'] else 0,axis=1) 
sia_list = df['sia_index'].to_list()
print(F"{sum(sia_list)/len(sia_list):.2%} - SentimentIntensityAnalyzer")

68.95% - SentimentIntensityAnalyzer


In [ ]:
new_review = "I'm in a hurry"
f = extract_features(new_review)
classifier.classify(f)


'neg'